<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


<B>The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site,</B> i.e., the initial position of rocket trajectories.<B> Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.</B>


<span style = 'color: green'><b>In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.</b></span>


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/ failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

<span style = 'color: green'><b>After completed the above tasks, you should be able to find some geographical patterns about launch sites.</b></span>


Let's first import required Python packages for this lab:


In [ ]:
#import piplite
#await piplite.install(['folium'])
#await piplite.install(['pandas'])

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)

## <span style= 'color: blue'> Task 1: Mark all launch sites on a map</span>

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
import requests
import io
from io import BytesIO

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
spacex_csv_file = BytesIO(response.content)
spacex_df = pd.read_csv(spacex_csv_file)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
# This line groups the DataFrame by 'Launch Site' and selects the first row in each group. So if there are multiple rows for 
# the same launch site (which is likely), it just takes the first occurrence.
# as_index=False: ensures 'Launch Site' stays as a regular column, not the index
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
# to create a folium Map object
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

<span style = 'color: blue'><b>We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,</b></span>

In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(
    nasa_coordinate,  # Tuple: (latitude, longitude)
    radius=1000,      # Radius in meters
    color='#d35400',  # Border color (a shade of orange)
    fill=True         # Fill the circle with color
 ).add_child(
    folium.Popup('NASA Johnson Space Center')  # Add a popup label on click
 )

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
# Place a text label (not a pin) at the same location
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label (Uses HTML (DivIcon) to render the name "NASA JSC" directly on the map in orange.)
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0), 
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )

# Adding to Map: Adds both the circle and the label marker to the existing Folium map object site_map
site_map.add_child(circle)
site_map.add_child(marker)

<span style = 'color: green'><b>and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.</b></span>

<span style = 'color: purple'><b>Now, let's add a circle for each launch site in data frame `launch_sites`</b></span>


<span style = 'color: green'><b>Note: after calling ".drop_duplicates()", the index remains non-sequential, so we use Use .iloc[i]['col'] instead of ['col'][i]. It allows you to select data by row and column position, rather than by label or index name.</b></span>  

In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# We have duplicate coordinates or rows in our DataFrame, so drop duplicate rows based on coordinates and launch site name.
unique_sites_df = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values and Launch site name as a popup label
for i in range (unique_sites_df.shape[0]):
    # add a Circle object based on its coordinate (Lat, Long) values
    site_name = unique_sites_df.iloc[i]['Launch Site']
    circle = folium.Circle(
        [unique_sites_df.iloc[i]['Lat'], unique_sites_df.iloc[i]['Long']],  # coordinate of esch  launch site (latitude, longitude)
        radius=1000,      # Radius in meters
        color='#d35400',  # Border color (a shade of orange)
        fill=True         # Fill the circle with color
    ).add_child(
        folium.Popup(f"{site_name} Space Center")  # Add a popup label on click
    )
    # Place a text label (not a pin) at the same location
    marker = folium.map.Marker(
    [unique_sites_df.iloc[i]['Lat'], unique_sites_df.iloc[i]['Long']],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0), 
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site_name}</b></div>'
        )
    )
    # Adds both the circle and the label marker to the existing Folium map object site_map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Show the map in Jupyter Notebook
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


<span style = 'color: purple'><b>Now, you can explore the map by zoom-in/out the marked areas, and try to answer the following questions:</b></span>

*   <span style = 'color: purple'><br><b>Are all launch sites in proximity to the Equator line?</b></br></span>
<span style = 'color: green'><b>No, not all launch sites are close to the equator, but most are located in the southern half of the continental U.S., which is closer to the equator than northern regions.</b></span>
<b>Reasoning: The Equator is at 0° latitude</b>

<b>Why closer to the Equator is better:</b>
The Earth's rotational speed is greatest at the equator.

*   <span style = 'color: purple'><br><b>Are all launch sites in very close proximity to the coast?</b></br></span>
<span style = 'color: green'><b>Yes, all launch sites shown are located near coastlines.</b></span>
<b>Reasoning: From the map:</b>

<b>Why coastal locations are preferred:</b>
  - Safety: Launch paths often go over open ocean, minimizing risk to populated areas if something goes wrong.
  - Debris management: In case of rocket stage separation or failure, debris falls into the ocean.
  - Range: Easier to create long, unobstructed eastward launch paths over water (especially from east-facing coastlines like Florida).

<span style = 'color: purple'><b>Also please try to explain your findings.</b></span>


## <span style= 'color: blue'> Task 2: Mark the success/ failed launches for each site on the map</span>


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


<span style = 'color: green'><b>Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`</b></span>

<span style = 'color: darkred'><b>Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.`</b></span>


<span style = 'color: green'><b>Let's first create a `MarkerCluster` object</b></span>

In [12]:
marker_cluster = MarkerCluster()

<span style = 'color: blue'><b>*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value</b></span>
 - <span style = 'color: blue'><b>Apply a function to check the value of `class` column</b></span>
 - <span style = 'color: blue'><b>If class=1, marker_color value will be green</b></span>
 - <span style = 'color: blue'><b> If class=0, marker_color value will be red</b></span>

In [13]:
def set_marker_color (df):
    df['marker_color'] = df['class'].apply(lambda x: 'green' if  x == 1 else 'red')
    return df

            
spacex_df = set_marker_color(spacex_df)

<span style = 'color: blue'><b>*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`</b></span>

In [14]:
# Create marker cluster and add to current site_map
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

# For each row in the DataFrame, add a color-coded marker to the cluster
for index, record in spacex_df.iterrows():
    lat = record['Lat']
    lon = record['Long']
    site = record['Launch Site']
    outcome = record['class']
    color = record['marker_color']  # already 'green' or 'red'
    # create a Marker object with its coordinate
    # and customize the Marker's icon property to indicate if this launch was successed or failed, 
    # e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
    # Create a marker with color-coded icon
    marker = folium.Marker(
        location=[lat, lon],
        popup=f"{site} - {'Success' if outcome == 1 else 'Failure'}",
        icon=folium.Icon(color=color, icon_color='white', icon='rocket', prefix='fa')  # Font Awesome rocket icon
    )
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


<span style = 'color: green'><b>From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.</b></span>


## <span style= 'color: blue'>TASK 3: Calculate the distances between a launch site to its proximities</span>


Next, we need to explore and analyze the proximities of launch sites.

<span style= 'color: purple'><b>Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)</b></span>

In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

<span style= 'color: purple'><b>Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.</b></span>


In [16]:
from math import sin, cos, sqrt, atan2, radians

# This fuction implements the Haversine formula, which calculates the great-circle distance between two points on the Earth's 
# surface — the shortest distance over the globe,as the crow fliescalculates.

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1) # radians(lat1) converts an angle from degrees to radians
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c 
    return distance


<span style= 'color: blue'><b>*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.</b></span>

In [17]:

# Coordinates of the nearest coastline point using MousePosition,e.g.,:
coastline_lat = 28.56367
coastline_lon = -80.57163

# Coordinates of the launch site (example: CCAFS LC-40)
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Calculate the distance using the calculate_distance function (Haversine function)
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)


# Add a marker at the coastline point with distance info
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance)
    )
)

# Add the marker to your folium map
site_map.add_child(distance_marker)

# To show the map in Jupyter:
site_map

<span style= 'color: blue'><b>*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point</b></span>


In [18]:
# Draw a PolyLine between a launch site and the coastline 

# Define coordinates: from launch site to coastline point
coordinates = [
    [launch_site_lat, launch_site_lon],     # e.g. Launch Site: CCAFS LC-40
    [coastline_lat, coastline_lon]          # e.g. Nearest coastline point
]

# Create the PolyLine
lines = folium.PolyLine(
    locations=coordinates,  # list of [lat, lon] pairs
    color='green',           # optional: color of the line
    weight=2                # thickness of the line
)

# Add line to the map
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


<span style= 'color: blue'><b>*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first</b></span>


<span style= 'color: purple'><b>A railway map symbol may look like this:</b></span>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


<span style= 'color: purple'><b>A highway map symbol may look like this:</b></span>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


<span style= 'color: purple'><b>A city map symbol may look like this:</b></span>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [21]:
# Create a marker with distance to a closest railway
# Coordinates of the closest city using MousePosition,e.g., Titusville:
city_lat = 28.61014
city_lon = -80.81088

# Coordinates of the launch site (example: KSC LC-39A
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Calculate the distance using the calculate_distance function
distance = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)


# Add a marker at the coastline point with distance info
distance_marker = folium.Marker(
    location=[city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance)
    )
)

# Add the marker to your folium map
site_map.add_child(distance_marker)

# To show the map in Jupyter:
site_map

# Draw a PolyLine between a launch site and the city
# Define coordinates: from launch site to city point
coordinates = [
    [launch_site_lat, launch_site_lon],     # e.g. Launch Site: KSC LC-39A
    [city_lat, city_lon]          # e.g. Titusville
]

# Create the PolyLine
lines = folium.PolyLine(
    locations=coordinates,  # list of [lat, lon] pairs
    color='green',           # optional: color of the line
    weight=2                # thickness of the line
)

# Add line to the map
site_map.add_child(lines)

In [20]:
# Create a marker with distance to a closest railway
# Coordinates of the nearest railway using MousePosition,e.g.,:
railway_lat = 28.57035
railway_lon = -80.80343

# Coordinates of the launch site (example: KSC LC-39A
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Calculate the distance using the calculate_distance function
distance = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)


# Add a marker at the coastline point with distance info
distance_marker = folium.Marker(
    location=[railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance)
    )
)

# Add the marker to your folium map
site_map.add_child(distance_marker)

# To show the map in Jupyter:
site_map


#  Draw a PolyLine between a launch site and the railway
# Define coordinates: from launch site to railway point
coordinates = [
    [launch_site_lat, launch_site_lon],     # e.g. Launch Site: KSC LC-39A
    [railway_lat, railway_lon]          # e.g. Nearest railway point
]

# Create the PolyLine
lines = folium.PolyLine(
    locations=coordinates,  # list of [lat, lon] pairs
    color='green',           # optional: color of the line
    weight=2                # thickness of the line
)

# Add line to the map
site_map.add_child(lines)

In [22]:
# Create a marker with distance to a closest highway
# Coordinates of the closest highway using MousePosition,e.g.,:
highway_lat = 28.57389
highway_lon = -80.65545

# Coordinates of the launch site (example: KSC LC-39A
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Calculate the distance using the calculate_distance function
distance = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)


# Add a marker at the coastline point with distance info
distance_marker = folium.Marker(
    location=[highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance)
    )
)

# Add the marker to your folium map
site_map.add_child(distance_marker)

# To show the map in Jupyter:
site_map


#  Draw a PolyLine between a launch site and the highway
# Define coordinates: from launch site to highway
coordinates = [
    [launch_site_lat, launch_site_lon],     # e.g. Launch Site: KSC LC-39A
    [highway_lat, highway_lon]          # e.g. Nearest highway
]

# Create the PolyLine
lines = folium.PolyLine(
    locations=coordinates,  # list of [lat, lon] pairs
    color='green',           # optional: color of the line
    weight=2                # thickness of the line
)

# Add line to the map
site_map.add_child(lines)

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

<b>Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.</b>


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
